# Import

In [3]:
import os
import pyaudio

# Current Proposed Methods
- Low Pass filter and find peaks in bass
- Calculate the entropy of music
- Perform filtering on all of it
# Previous Publications of interest
- https://www.researchgate.net/publication/228987153_Detection_of_Spirited_Incidental_Music_in_Movies

# Idea 1: Calculate Peaks by filtering amplitude

## Load song

## Get Amplitude Data

## Perform Thresholding